In [76]:
using CSV, DataFrames, Dates

using PyCall
using CairoMakie

@pyimport powerlaw as powlaw
so = pyimport("scipy.optimize")

PyObject <module 'scipy.optimize' from '/home/anicolin/.local/lib/python3.10/site-packages/scipy/optimize/__init__.py'>

In [81]:
data_in_question = "moonquakes_all_stations"
variable = "amplitude"

path = "./data/"
filepath = "./data/" * data_in_question * ".csv"
df = CSV.read(filepath, DataFrame);

mkpath("./results/$(data_in_question)")

"./results/moonquakes_all_stations"

In [83]:
function waiting_times(df, delta, datetime, variable)
    wtid = []
    for (i,k) in enumerate(df[!,variable])
        first_event = df[!,datetime][i]
        for (j,x) in enumerate(df[i+1:end, variable])
            if x >= (k+delta)
                second_event = df[!,datetime][i+j]
                push!(wtid, trunc(Int, Dates.value(second_event - first_event) / 150000) )
                break
            end
        end
    end
    return wtid
end

waiting_times (generic function with 1 method)

In [84]:
deltas = 0.5:0.5:5
datetime = "datetime"
for delta in deltas
    wt = waiting_times(df, delta, datetime, variable)
    wt_dataframe = DataFrame([wt], ["wt"])
    CSV.write("./results/$(data_in_question)/wt_$(data_in_question)_delta_$(delta).csv", wt_dataframe, delim=",")
end

In [91]:
wt_alpha = []
wt_sigma = []
wt_xmin = []
wt_KS = []



deltas = collect(0.1:0.1:1.5)
for delta in deltas

    wt_df = CSV.read("./results/$(data_in_question)/wt_$(data_in_question)_delta_$(delta).csv", DataFrame)
    wt = wt_df[!,:wt]

    mkpath("./results/$(data_in_question)/")


    # fit_tsallis = pyeval("""lambda fit: lambda a, b, c, d: fit(a, b, c, d)""")
    # @. tsallis_ccdf(x, α, λ, c) = c*((1+x/(λ))^(-α))
    # popt_tsallis, pcov_tsallis = so.curve_fit(fit_tsallis((x, α, λ, c)->tsallis_ccdf(x, α, λ, c)), x_ccdf_original_data, y_ccdf_original_data, bounds=(0, Inf), maxfev=3000)
    # println("alpha= ", popt_tsallis[1],"\nlambda= ", popt_tsallis[2], "\nc= ", popt_tsallis[3])

    # alpha = round(popt_tsallis[1], digits=4)
    # lambda = round(popt_tsallis[2], digits=4)
    # c = round(popt_tsallis[3], digits=4)

    # Powerlaw fit
    fit = powlaw.Fit(wt);
    alpha = fit.alpha
    xmin = fit.xmin
    sigma = fit.sigma
    KS = fit.power_law.KS(data=wt)

    push!(wt_alpha, alpha )
    push!(wt_sigma, sigma )
    push!(wt_xmin, xmin )
    push!(wt_KS, KS )

    x_ccdf, y_ccdf = fit.ccdf()

    # The fit (from theoretical power_law)
    fit_power_law = fit.power_law.plot_ccdf()[:lines][1]
    x_powlaw, y_powlaw = fit_power_law[:get_xdata](), fit_power_law[:get_ydata]()

    # Round up the data
    alpha = round(alpha, digits=2)
    sigma = round(sigma, digits=2)
    xmin = round(xmin, digits=4)
    KS = round(KS, digits=3)


    set_theme!(Theme(fonts=(; regular="CMU Serif")))

    markers=[:utriangle, :circle]
    colors=[:lightblue, :lightgreen]
    line_colors=[:midnightblue, :green]
    i=1

    #############################################################################################################################################################
    # THE PLOTS 
    fig = Figure(resolution = (700, 600), font= "CMU Serif") 
    ax1 = Axis(fig[1,1], xlabel = L"hours", ylabel = L"P", xscale=log10, yscale=log10, ylabelsize = 28,
        xlabelsize = 28, xgridstyle = :dash, ygridstyle = :dash, xtickalign = 1,
        xticksize = 5, ytickalign = 1, yticksize = 5 , xlabelpadding = 10, ylabelpadding = 10, xticklabelsize=22, yticklabelsize=22)
        
    # # CCDF of truncated data (fitted), the plot, (re-normed)
    ax2 = Axis(fig, bbox = BBox(150,400,116,330), xscale=log10, yscale=log10, xgridvisible = false, ygridvisible = false, xtickalign = 1,
        xticksize = 4, ytickalign = 1, yticksize = 4, xticklabelsize=16, yticklabelsize=16, backgroundcolor=:white)


    ########################################### ALL
    # CCDF of all data scattered 
    x_ccdf_original_data, y_ccdf_original_data = powlaw.ccdf(wt)


    sc1 = scatter!(ax1, x_ccdf_original_data, y_ccdf_original_data,
        color=(colors[1], 0.75), strokewidth=0.05, strokecolor=(line_colors[i], 0.8), marker=markers[i], markersize=12)

    # Fit through truncated data
    # Must shift the y values from the theoretical powerlaw by the values of y of original data, but cut to the length of truncated data
    ln1 = lines!(ax1, x_powlaw, y_ccdf_original_data[end-length(x_ccdf)] .* y_powlaw, label = L"\alpha=%$(alpha)\pm %$(sigma),\, x_{\mathrm{min}}=%$(xmin),\, \mathrm{KS}=%$(KS)",
        color=line_colors[1], linewidth=2.5) 

    ########################################### TRUNCATED
    sc2 = scatter!(ax2, x_ccdf, y_ccdf,
    color=(colors[1], 0.75), strokewidth=0.05, strokecolor=(line_colors[i], 0.8), marker=markers[i], markersize=10)

    # Fit through truncated data (re-normed)
    ln2 = lines!(ax2, x_powlaw, y_powlaw,
        color=line_colors[1], linewidth=2.5)
        
        
    # AXIS LEGEND
    axislegend(ax1, [sc1], 
    [L"\delta=%$(delta)"],
    position = :rt, backgroundcolor = (:grey90, 0.25), labelsize=18, titlesize=18);

    axislegend(ax1, position = :lt, backgroundcolor = (:grey90, 0.25), labelsize=18);

    ylims!(ax1, 10^(-6), 10^(0.7))

    save("./results/$(data_in_question)/wt_$(data_in_question)_delta_$(delta).png", fig, px_per_unit=5)

end

# results = DataFrame([deltas, wt_alpha, wt_sigma, wt_xmin, wt_KS], ["delta", "alpha", "sigma", "xmin", "KS"])
CSV.write("./results/$(data_in_question)/$(data_in_question).csv", results, delim=",", header=true);

In [ ]:
results = CSV.read("./results_$(data_in_question).csv", DataFrame)

set_theme!(Theme(fonts=(; regular="CMU Serif")))
cmap = :thermometer
fig = Figure(resolution = (1000, 400), font= "CMU Serif") 
xlabels = [L"\delta\,\text{[km/h]}",L"\delta\,\text{[km/h]}", ""]
ylabels = [ L"\alpha", L"x_{min}", ""]
ax = [Axis(fig[1, i], xlabel = xlabels[i], ylabel = ylabels[i], ylabelsize = 26,
    xlabelsize = 22, xgridstyle = :dash, ygridstyle = :dash, xtickalign = 1,
    xticksize = 5, ytickalign = 1, yticksize = 5 , xlabelpadding = 10, ylabelpadding = 10) for i in 1:2]
scatter!(ax[1],results.delta, results.alpha; color = results.KS, colormap = cmap,
    markersize = 13, marker = :circle, strokewidth = 0)
err = errorbars!(ax[1], results.delta, results.alpha, results.sigma; color = results.KS, colormap = cmap, 
whiskerwidth = 4)
xmin_scat = scatter!(ax[2],results.delta, results.xmin; color = results.KS, colormap = cmap,
    markersize = 13, marker = :circle, strokewidth = 0)
cbar = Colorbar(fig[1,3], xmin_scat, vertical = true, label="KS", flipaxis=false)
# Label(fig[1, 1, Top()], variable, fontsize=16,
#     padding=(0, 0, 15, 0))
save("./results_$(data_in_question)_par_dep.png", fig, px_per_unit=5)

fig